In [12]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
df = pd.read_parquet('D:/escritorio/STEAM/parquet/engineer.parquet')
df.head(2)

,item_id,item_name,playtime_forever,items_count,genres,app_name,price,developer,release_year,posted_year,recommend,review,sentiment_analysis,user_id
685189,4000,Garry's Mod,4974,20,Indie,Garry's Mod,9.99,Facepunch Studios,2006,2014,True,I recommend getting this game because it is su...,2,ignaciomena22
1067561,304930,Unturned,1822,15,Action,Unturned,0.00,Smartly Dressed Games,2017,2015,True,I've just started this game and already enjoyi...,2,76561198077820683


In [14]:
#Creo un nuevo dataframe con las columnas necesarias para el modelo
df1 = df[['item_id','app_name','genres']].copy()
df1

,item_id,app_name,genres
685189,4000,Garry's Mod,Indie
1067561,304930,Unturned,Action
360168,4000,Garry's Mod,Indie
95041,4000,Garry's Mod,Indie
1846618,730,Counter-Strike: Global Offensive,Action
...,...,...,...
2842694,6060,"Star Wars: Battlefront 2 (Classic, 2005)",Action
1065511,304930,Unturned,Action
2876324,113200,The Binding of Isaac,Action
2904524,219150,Hotline Miami,Action


In [15]:
df1.rename(columns={'genres': 'genres'},inplace=True)
df1['item_id'] = df['item_id'].astype(int)
df1.head(2)

,item_id,app_name,genres
685189,4000,Garry's Mod,Indie
1067561,304930,Unturned,Action


In [16]:
# Crea las variables dummi para la columna especificada
modelado = pd.get_dummies(df1, columns=['genres'], prefix="")
df_modelo = modelado.groupby(['item_id','app_name']).sum().reset_index()
df_modelo.sample(3)

,item_id,app_name,_Action,_Adventure,_Animation &amp; Modeling,_Casual,_Free to Play,_Indie,_RPG,_Racing,_Simulation,_Strategy,_Utilities
79,19830,Tom Clancy's Rainbow Six® 3 Gold,1,0,0,0,0,0,0,0,0,0,0
318,232750,Mars: War Logs,3,0,0,0,0,0,0,0,0,0,0
174,62000,Flight Control HD,2,0,0,0,0,0,0,0,0,0,0


In [17]:
df_modelo.to_parquet('D:/escritorio/STEAM/parquet/modelo.parquet')

In [18]:
# La función recibe un id de un juego y en base a este retorna una lista con juegos similares
def recomendacion_juego(id:int):
    
    # Crea un df con los datos de el id pasado como parametro
    juego = df_modelo[df_modelo['item_id'] == id]
    
    # En caso que el ID no esté en el df, muestra el siguiente mensaje:
    if juego.empty:
        return f"El juego con ID {id} no se encuentra."

    # Obtiene el índice del juego en el DataFrame.
    idx = juego.index[0]

    # Tamaño de la muestra
    muestra = min(2000, len(df_modelo))

    # Ajustamos la semilla aleatoria
    df_sample = df_modelo.sample(n=muestra, random_state=42)
    
    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([df_modelo.iloc[idx, 3:]], df_sample.iloc[:, 3:])
    
    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"Juegos similares para el juego con ID": id, "Juegos similares": similar_game_names}

# Ahora puedes ingresar el ID del juego que desees para obtener recomendaciones
recomendacion_juego(70)
# Por ejemplo, aquí se pasa otro ID como argumento

{'Juegos similares para el juego con ID': 70,
 'Juegos similares': ['Crusader Kings II',
  'Tomb Raider: Anniversary',
  'Robocraft',
  'Total War Battles: KINGDOM',
  'X-COM: Enforcer']}

In [19]:
recomendacion_juego(2700)

{'Juegos similares para el juego con ID': 2700,
 'Juegos similares': ['Wings of Prey',
  'Silent Hunter®: Wolves of the Pacific',
  'Tropico Reloaded',
  'Stronghold Crusader 2',
  'RollerCoaster Tycoon® 3: Platinum']}

In [20]:
recomendacion_juego(8000)

{'Juegos similares para el juego con ID': 8000,
 'Juegos similares': ['The Lord of the Rings Online™',
  'Crusader Kings II',
  'Tomb Raider: Anniversary',
  'Robocraft',
  'Total War Battles: KINGDOM']}